In [1]:
import time
import pandas as pd
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from joblib import dump,load
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# ОПИСАНИЕ 

## итак были добавлены методы get_train_dataset , get_dataset, percent_missing, clear_data . Были изменены все остальные этот ноубук по сути шаблон для нашей дальнейшей работы , обьясню на пальцах что тут к чему:

## Все что не автоматизировано это загрузка данных собственоо загружаем x_train y_train (нам потребуются для обучения) далее в методах класса get_train_dataset и clear_data предусмотрена их обработка не для частых случаев конечно но в целом сойдет для большинтсва get_train_dataset используется только один раз при обучении merge их и тп все предусмотрено в этих двух методах 

## МЕТОД GET_DATASET подключен к predict так что при загрузке новых данных мы просто посылаем их туда и там все обрабатывается и выдаются predict-ы в виде словаря (см ниже)

## МОДЕЛИ RIDGE И ELASTICNET ПРОСТО ДЛЯ ПРИМЕРА вы будете тестить другие при различных комбинациях параметров в param_grid (см train_model) просто заменяете на другие меняете param_grid и смотрите

## ПОРЯДОК РАБОТЫ при первичном запуске напишите следующее:

## air = AircraftModel() СОЗДАЕМ ЭКСЕМПЛЯР КЛАССА
## df = air.get_train_dataset(X_train, y_train) генерируем train_data только единственный раз при обучении! далее не используется
## df = air.clear_data(df) чистим данные
## air.train_model(df) обучаем и при этом в методе возвращаются метрики (очень удобно в виде словаря )
## на данном этапе первичный запуск завершен модель обучена !

## ПОРЯДОК РАБОТЫ ПРИ ЗАПУСКЕ НА НОВЫХ ДАННЫХ
## ЗАГРУЖАЕМ ИХ X_test = pd.read_csv('X_test.csv',parse_dates=['reportts'])
## predict = air.predict(X_test) 
## И ВСЕ! В PREDICT БУДЕТ СЛОВАРЬ С ПРЕДСКАЗАНИЯМИ (air- эксемпляр класса создан ранее )




In [2]:
X_train = pd.read_csv('X_train.csv', parse_dates=['reportts'])


In [3]:
y_train = pd.read_csv('y_train.csv', parse_dates=['reportts'])


In [4]:
X_test = pd.read_csv('X_test.csv',parse_dates=['reportts'])

In [5]:
class AircraftModel:
   
    def __init__(self):
        self.models = {}
        
    def get_train_dataset(self, X_train, y_train) :
        df = X_train.merge(y_train, on=['acnum', 'pos', 'reportts'])
        cols = ['egt', 'n1a', 'n2a', 'nf', 'ff', 'mn', 't2', 'tat', 'oat', 'alt',
                'p2e','wai', 'nai', 'prv', 'hpv', 'xf', 'acnum', 'egtm'] 
        dataset = df[cols]
        return dataset 
    
    
    def get_dataset(self, X_test):
        cols = ['egt', 'n1a', 'n2a', 'nf', 'ff', 'mn', 't2', 'tat', 'oat', 'alt',
                'p2e','wai', 'nai', 'prv', 'hpv', 'xf', 'acnum'] 
        dataset = df[cols]
        return dataset
    def train_model(self, df):
        metrics = {}
        for acnum in df['acnum'].unique():
            model_df = df[df['acnum'] == acnum] 
            valid_data = model_df.drop('acnum', axis=1) 
            X_train, X_test, y_train, y_test = train_test_split(valid_data.drop(['egtm'], axis=1), valid_data['egtm'], test_size=0.25) 
            
            if acnum == 'VQ-BGU': 
                model = Ridge() 
                param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]} 
                grid_model = GridSearchCV(estimator=model, param_grid=param_grid, 
                                          scoring='neg_mean_squared_error', cv=5, error_score='raise') 
                grid_model.fit(X_train, y_train) 
                self.models[acnum] = grid_model.best_estimator_ 
                
            elif acnum == 'VQ-BDU':
                model = ElasticNet() 
                param_grid = {'alpha': [0.1, 1, 5, 10, 50, 100], 'l1_ratio': [.1, .5, .7, .9, .95, .99, 1]} 
                grid_model = GridSearchCV(estimator=model, param_grid=param_grid, 
                                          scoring='neg_mean_squared_error', cv=5, error_score='raise') 
                grid_model.fit(X_train, y_train) 
                self.models[acnum] = grid_model.best_estimator_ 

            else: 
                raise ValueError(f"No such aircraft{acnum}") 
            
            predictions = self.models[acnum].predict(X_test)
            mae = mean_absolute_error(y_test, predictions)
            rmse = np.sqrt(mean_squared_error(y_test, predictions))
            mape_score = mean_absolute_percentage_error(y_test, predictions)
            r2score = r2_score(y_test, predictions)

            metrics[acnum] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape_score, 'R2 Score': r2score}

        return metrics
    
    def predict(self, df): 
        df_prepared = self.get_dataset(df)
        df_cleaned = self.clear_data(df_prepared)
        
        results = {}
        for acnum in df_prepared['acnum'].unique():
            model = self.models.get(acnum)
            if model:
                X_test = df_prepared[df_prepared['acnum'] == acnum].drop('acnum', axis=1)
                predictions = model.predict(X_test)
                results[acnum] = predictions
            else:
                raise ValueError(f"No predictions stored for aircraft model {acnum}")
        return results
    def percent_missing(self, df):
        percent_nan = (df.isnull().sum() / len(df)) * 100
        return percent_nan[percent_nan > 0].sort_values()

    def clear_data(self,df):
        percent_miss = self.percent_missing(df)
        columns_to_drop = percent_miss[percent_miss > 95].index
        df = df.drop(columns=columns_to_drop)
        columns_to_fill = percent_miss[(percent_miss > 0) & (percent_miss <= 5)].index
        for column in columns_to_fill:
            df[column].fillna(df[column].median(), inplace=True)

        return df

        
        
#         df_encoded = pd.get_dummies(df, columns=['acnum'])
        
         

In [6]:
air = AircraftModel()

In [7]:
df = air.get_train_dataset(X_train, y_train)

In [8]:
df = air.clear_data(df)

In [9]:
air.train_model(df)

/home/fltww/anaconda3/envs/My_First_ML_Course/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.659e+00, tolerance: 3.874e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/fltww/anaconda3/envs/My_First_ML_Course/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.444e+00, tolerance: 3.905e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/fltww/anaconda3/envs/My_First_ML_Course/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might wan

/home/fltww/anaconda3/envs/My_First_ML_Course/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.418e+00, tolerance: 3.874e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/fltww/anaconda3/envs/My_First_ML_Course/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.117e+00, tolerance: 3.905e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/fltww/anaconda3/envs/My_First_ML_Course/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might wan

{'VQ-BGU': {'MAE': 2.42841431386003,
  'RMSE': 3.1162334640037592,
  'MAPE': 0.07893718515656817,
  'R2 Score': 0.7172604153236484},
 'VQ-BDU': {'MAE': 2.3639113967005656,
  'RMSE': 3.0299368371958497,
  'MAPE': 0.09657824410859532,
  'R2 Score': 0.7989737227540729}}

In [21]:
air.models

{'VQ-BGU': Ridge(alpha=0.01), 'VQ-BDU': ElasticNet(alpha=0.1, l1_ratio=1)}

In [10]:
predict = air.predict(X_test)

In [11]:
predict

{'VQ-BGU': array([32.43139628, 32.76746162, 36.26197797, ..., 24.73218687,
        25.72675043, 25.21857952]),
 'VQ-BDU': array([39.20688323, 39.31591848, 38.02108171, ..., 20.27301322,
        22.16330675, 23.62776299])}

# ЕСЛИ ЕСТЬ КОРРИКТИРОВКИ ИЛИ ПРЕДЛОЖЕНИЯ пишем меняем не стесняемся 